# Spark Lab
This lab will demonstrate how to perform web server log analysis with Spark. Log data is a very large, common data source and contains a rich set of information.  It comes from many sources, such as web, file, and compute servers, application logs, user-generated content,  and can be used for monitoring servers, improving business and customer intelligence, building recommendation systems, fraud detection, and much more. This lab will show you how to use Spark on real-world text-based production logs and fully harness the power of that data.

### Apache Web Server Log file format
The log files that we use for this assignment are in the [Apache Common Log Format (CLF)](http://httpd.apache.org/docs/1.3/logs.html#common) format. The log file entries produced in CLF will look something like this:
`127.0.0.1 - - [01/Aug/1995:00:00:01 -0400] "GET /images/launch-logo.gif HTTP/1.0" 200 1839`
 
Each part of this log entry is described below.
* **`127.0.0.1`:** this is the IP address (or host name, if available) of the client (remote host) which made the request to the server. 
* **`-`:** the "hyphen" in the output indicates that the requested piece of information (user identity from remote machine) is not available.
* **`-`:** the "hyphen" in the output indicates that the requested piece of information (user identity from local logon) is not available. 
* **`[01/Aug/1995:00:00:01 -0400]`:** the time that the server finished processing the request. The format is:
`[day/month/year:hour:minute:second timezone]`.
* **`"GET /images/launch-logo.gif HTTP/1.0"`:** this is the first line of the request string from the client. It consists of a three components: the request method (e.g., `GET`, `POST`, etc.), the endpoint, and the client protocol version. 
* **`200`:** this is the status code that the server sends back to the client. This information is very valuable, because it reveals whether the request resulted in a successful response (codes beginning in 2), a redirection (codes beginning in 3), an error caused by the client (codes beginning in 4), or an error in the server (codes beginning in 5). The full list of possible status codes can be found in the HTTP specification ([RFC 2616](https://www.ietf.org/rfc/rfc2616.txt) section 10).
* **`1839`:** the last entry indicates the size of the object returned to the client, not including the response headers. If no content was returned to the client, this value will be "-" (or sometimes 0).
 
Using the CLF as defined above, we create a regular expression pattern to extract the nine fields of the log line. The function returns a pair consisting of a Row object and 1. If the log line fails to match the regular expression, the function returns a pair consisting of the log line string and 0. A '-' value in the content size field is cleaned up by substituting it with 0. The function converts the log line's date string into a `Cal` object using the given `parseApacheTime` function. We, then, create the primary RDD and  we'll use in the rest of this assignment. We first load the text file and convert each line of the file into an element in an RDD. Next, we use `map(parseApacheLogLine)` to apply the parse function to each element and turn each line into a pair `Row` object. Finally, we cache the RDD in memory since we'll use it throughout this notebook. The log file is available at `data/apache/apache.log`.

In [1]:
import scala.util.matching
import org.apache.spark.rdd.RDD

case class Cal(year: Int, month: Int, day: Int, hour: Int, minute: Int, second: Int)                

case class Row(host: String, clientID: String, userID: String, dateTime: Cal, method: String, endpoint: String,
               protocol: String, responseCode: Int, contentSize: Long)                

val month_map = Map("Jan" -> 1, "Feb" -> 2, "Mar" -> 3, "Apr" -> 4, "May" -> 5, "Jun" -> 6, "Jul" -> 7, "Aug" -> 8,
                    "Sep" -> 9, "Oct" -> 10, "Nov" -> 11, "Dec" -> 12)

//------------------------------------------------
// A regular expression pattern to extract fields from the log line
val APACHE_ACCESS_LOG_PATTERN = """^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)\s*(\S*)\s*" (\d{3}) (\S+)""".r

//------------------------------------------------
def parseApacheTime(s: String): Cal = {
    return Cal(s.substring(7, 11).toInt, month_map(s.substring(3, 6)), s.substring(0, 2).toInt, 
               s.substring(12, 14).toInt, s.substring(15, 17).toInt, s.substring(18, 20).toInt)
}

//------------------------------------------------
def parseApacheLogLine(logline: String): (Either[Row, String], Int) = {
    val ret = APACHE_ACCESS_LOG_PATTERN.findAllIn(logline).matchData.toList
    if (ret.isEmpty)
        return (Right(logline), 0)

    val r = ret(0)
    val sizeField = r.group(9)

    var size: Long = 0
    if (sizeField != "-")
        size = sizeField.toLong

    return (Left(Row(r.group(1), r.group(2), r.group(3), parseApacheTime(r.group(4)), r.group(5), r.group(6),
                     r.group(7), r.group(8).toInt, size)), 1)
}

//------------------------------------------------
def parseLogs(): (RDD[(Either[Row, String], Int)], RDD[Row], RDD[String]) = {
    val fileName = "data/apache/apache.log"
    
    val parsedLogs = sc.textFile(fileName).map(parseApacheLogLine).cache()
    val accessLogs = parsedLogs.filter(x => x._2 == 1).map(x => x._1.left.get)
    val failedLogs = parsedLogs.filter(x => x._2 == 0).map(x => x._1.right.get)

    val failedLogsCount = failedLogs.count()
    
    if (failedLogsCount > 0) {
        println(s"Number of invalid logline: $failedLogs.count()")
        failedLogs.take(20).foreach(println)
    }
    println(s"Read ${parsedLogs.count()} lines, successfully parsed ${accessLogs.count()} lines, and failed to parse ${failedLogs.count()}")
    
    return (parsedLogs, accessLogs, failedLogs)
}

Intitializing Scala interpreter ...

Spark Web UI available at http://e49630fae5e5:4040
SparkContext available as 'sc' (version = 2.4.4, master = local[*], app id = local-1569022712325)
SparkSession available as 'spark'


import scala.util.matching
import org.apache.spark.rdd.RDD
defined class Cal
defined class Row
month_map: scala.collection.immutable.Map[String,Int] = Map(Nov -> 11, Jul -> 7, Mar -> 3, Jan -> 1, Oct -> 10, Dec -> 12, Feb -> 2, May -> 5, Apr -> 4, Aug -> 8, Sep -> 9, Jun -> 6)
APACHE_ACCESS_LOG_PATTERN: scala.util.matching.Regex = ^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)\s*(\S*)\s*" (\d{3}) (\S+)
parseApacheTime: (s: String)Cal
parseApacheLogLine: (logline: String)(Either[Row,String], Int)
parseLogs: ()(org.apache.spark.rdd.RDD[(Either[Row,String], Int)], org.apache.spark.rdd.RDD[Row], org.apache.spark.rdd.RDD[String])


In [2]:
val (parsedLogs, accessLogs, failedLogs) = parseLogs()

Read 1043177 lines, successfully parsed 1043177 lines, and failed to parse 0


parsedLogs: org.apache.spark.rdd.RDD[(Either[Row,String], Int)] = MapPartitionsRDD[2] at map at <console>:56
accessLogs: org.apache.spark.rdd.RDD[Row] = MapPartitionsRDD[4] at map at <console>:57
failedLogs: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[6] at map at <console>:58


### Sample Analyses on the Web Server Log File
Let's compute some statistics about the sizes of content being returned by the web server. In particular, we'd like to know what are the average, minimum, and maximum content sizes. We can compute the statistics by applying a `map` to the `accessLogs` RDD. The given function to the `map` should extract the `contentSize` field from the RDD. The `map` produces a new RDD, called `contentSizes`, containing only the `contentSizes`. To compute the minimum and maximum statistics, we can use `min()` and `max()` functions on the new RDD. We can compute the average statistic by using the `reduce` function with a function that sums the two inputs, which represent two elements from the new RDD that are being reduced together. The result of the `reduce()` is the total content size from the log and it is to be divided by the number of requests as determined using the `count()` function on the new RDD. As the result of executing the following box, you should get the below result:
```
Content Size Avg: 17531, Min: 0, Max: 3421948
```

In [3]:
// Calculate statistics based on the content size.
val contentSizes = accessLogs.map(_.contentSize).cache()
println("Content Size Avg: " + contentSizes.reduce(_ + _) / contentSizes.count() + ", Min: " + contentSizes.min + ", Max: " + contentSizes.max)

Content Size Avg: 17531, Min: 0, Max: 3421948


contentSizes: org.apache.spark.rdd.RDD[Long] = MapPartitionsRDD[7] at map at <console>:30


Next, lets look at the "response codes" that appear in the log. As with the content size analysis, first we create a new RDD that contains the `responseCode` field from the `accessLogs` RDD. The difference here is that we will use a *pair tuple* instead of just the field itself (i.e., (response code, 1)). Using a pair tuple consisting of the response code and 1 will let us count the number of of records with a particular response code. Using the new RDD `responseCodes`, we perform a `reduceByKey` function that applys a given function to each element, pairwise with the same key. Then, we cache the resulting RDD and create a list by using the `take` function. Once you run the code below, you should receive the following results:
```
Found 7 response codes
Response Code Counts: (404,6185) (200,940847) (304,79824) (500,2) (501,17) (302,16244) (403,58)
```

In [4]:
// extract the response code for each record and make pair of (response code, 1)
val responseCodes = accessLogs.map(x => (x.responseCode, 1))

// count the number of records for each key
val responseCodesCount = responseCodes.reduceByKey(_ + _).cache()

// take the first 100 records
val responseCodesCountList = responseCodesCount.take(100)

println("Found " + responseCodesCountList.length + " response codes")
print("Response Code Counts: ")
responseCodesCountList.foreach(x => print(x + " "))

Found 7 response codes
Response Code Counts: (404,6185) (200,940847) (304,79824) (500,2) (501,17) (302,16244) (403,58) 

responseCodes: org.apache.spark.rdd.RDD[(Int, Int)] = MapPartitionsRDD[8] at map at <console>:29
responseCodesCount: org.apache.spark.rdd.RDD[(Int, Int)] = ShuffledRDD[9] at reduceByKey at <console>:32
responseCodesCountList: Array[(Int, Int)] = Array((404,6185), (200,940847), (304,79824), (500,2), (501,17), (302,16244), (403,58))


Let's look at "hosts" that have accessed the server multiple times (e.g., more than 10 times). First we create a new RDD to keep the `host` field from the `accessLogs` RDD using a pair tuple consisting of the host and 1 (i.e., (host, 1)), which will let us count how many records were created by a particular host's request. Using the new RDD, we perform a `reduceByKey` function with a given function to add the two values. We then filter the result based on the count of accesses by each host (the second element of each pair) being greater than 10. Next, we extract the host name by performing a `map` to return the first element of each pair. Finally, we extract 20 elements from the resulting RDD. The result should be as below:
```
Any 20 hosts that have accessed more then 10 times: 
ix-aug-ga1-13.ix.netcom.com 
n1043347.ksc.nasa.gov 
d02.as1.nisiq.net 
192.112.22.82 
anx3p4.trib.com 
198.215.127.2 
198.77.113.34 
crc182.cac.washington.edu 
telford-107.salford.ac.uk 
universe6.barint.on.ca 
gatekeeper.homecare.com 
157.208.11.7 
unknown.edsa.co.za 
onyx.southwind.net 
ppp-hck-2-12.ios.com 
ix-lv5-04.ix.netcom.com 
f-umbc7.umbc.edu 
cs006p09.nam.micron.net 
dd22-025.compuserve.com 
hak-lin-kim.utm.edu 
```

In [5]:
// extract the host field for each record and make pair of (host, 1)
val hosts = accessLogs.map(x => (x.host, 1))

// count the number of records for each key
val hostsCount = hosts.reduceByKey(_ + _)

// keep the records with the count greater than 10
val hostMoreThan10 = hostsCount.filter(x => x._2 > 10)

// take the first 100 records
val hostsPick20 = hostMoreThan10.map(x => x._1).take(20)

println("Any 20 hosts that have accessed more then 10 times: ")
hostsPick20.foreach(println)

Any 20 hosts that have accessed more then 10 times: 
ix-aug-ga1-13.ix.netcom.com
n1043347.ksc.nasa.gov
d02.as1.nisiq.net
192.112.22.82
anx3p4.trib.com
198.215.127.2
198.77.113.34
crc182.cac.washington.edu
telford-107.salford.ac.uk
universe6.barint.on.ca
gatekeeper.homecare.com
157.208.11.7
unknown.edsa.co.za
onyx.southwind.net
ppp-hck-2-12.ios.com
ix-lv5-04.ix.netcom.com
f-umbc7.umbc.edu
cs006p09.nam.micron.net
dd22-025.compuserve.com
hak-lin-kim.utm.edu


hosts: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[10] at map at <console>:29
hostsCount: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[11] at reduceByKey at <console>:32
hostMoreThan10: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[12] at filter at <console>:35
hostsPick20: Array[String] = Array(ix-aug-ga1-13.ix.netcom.com, n1043347.ksc.nasa.gov, d02.as1.nisiq.net, 192.112.22.82, anx3p4.trib.com, 198.215.127.2, 198.77.113.34, crc182.cac.washington.edu, telford-107.salford.ac.uk, universe6.barint.on.ca, gatekeeper.homecare.com, 157.208.11.7, unknown.edsa.co.za, onyx.southwind.net, ppp-hck-2-12.ios.com, ix-lv5-04.ix.netcom.com, f-umbc7.umbc.edu, cs006p09.nam.micron.net, dd22-025.compuserve.com, hak-lin-kim.utm.edu)


For the final example, we'll look at the top endpoints (URIs) in the log. To determine them, we first create a new RDD to extract the `endpoint` field from the `accessLogs` RDD using a pair tuple consisting of the endpoint and 1 (i.e., (endpoint, 1)), which will let us count how many records were created by a particular host's request. Using the new RDD, we perform a `reduceByKey` to add the two values. We then extract the top 10 endpoints by performing a `takeOrdered` with a value of 10 and a function that multiplies the count (the second element of each pair) by -1 to create a sorted list with the top endpoints at the bottom. Here is the result:
```
Top ten endpoints: 
(/images/NASA-logosmall.gif,59737)
(/images/KSC-logosmall.gif,50452)
(/images/MOSAIC-logosmall.gif,43890)
(/images/USA-logosmall.gif,43664)
(/images/WORLD-logosmall.gif,43277)
(/images/ksclogo-medium.gif,41336)
(/ksc.html,28582)
(/history/apollo/images/apollo-logo1.gif,26778)
(/images/launch-logo.gif,24755)
(/,20292)
```

In [6]:
// extract the endpoint for each record and make pair of (endpoiint, 1)
val endpoints = accessLogs.map(x => (x.endpoint, 1))

// count the number of records for each key
val endpointCounts = endpoints.reduceByKey(_ + _)

// extract the top 10 
val topEndpoints = endpointCounts.takeOrdered(10)(Ordering[Int].on(x => x._2 * -1))

println("Top ten endpoints: ")
topEndpoints.foreach(println)

Top ten endpoints: 
(/images/NASA-logosmall.gif,59737)
(/images/KSC-logosmall.gif,50452)
(/images/MOSAIC-logosmall.gif,43890)
(/images/USA-logosmall.gif,43664)
(/images/WORLD-logosmall.gif,43277)
(/images/ksclogo-medium.gif,41336)
(/ksc.html,28582)
(/history/apollo/images/apollo-logo1.gif,26778)
(/images/launch-logo.gif,24755)
(/,20292)


endpoints: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[14] at map at <console>:29
endpointCounts: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[15] at reduceByKey at <console>:32
topEndpoints: Array[(String, Int)] = Array((/images/NASA-logosmall.gif,59737), (/images/KSC-logosmall.gif,50452), (/images/MOSAIC-logosmall.gif,43890), (/images/USA-logosmall.gif,43664), (/images/WORLD-logosmall.gif,43277), (/images/ksclogo-medium.gif,41336), (/ksc.html,28582), (/history/apollo/images/apollo-logo1.gif,26778), (/images/launch-logo.gif,24755), (/,20292))


### Analyzing Web Server Log File
What are the top ten endpoints which did not have return code 200? Create a sorted list containing top ten endpoints and the number of times that they were accessed with non-200 return code. Think about the steps that you need to perform to determine which endpoints did not have a 200 return code, how you will uniquely count those endpoints, and sort the list. You should receive the following result:
```
Top ten failed URLs: 
(/images/NASA-logosmall.gif,8761)
(/images/KSC-logosmall.gif,7236)
(/images/MOSAIC-logosmall.gif,5197)
(/images/USA-logosmall.gif,5157)
(/images/WORLD-logosmall.gif,5020)
(/images/ksclogo-medium.gif,4728)
(/history/apollo/images/apollo-logo1.gif,2907)
(/images/launch-logo.gif,2811)
(/,2199)
(/images/ksclogosmall.gif,1622)
```

In [7]:
// keep the logs with error code not 200
val not200 = accessLogs.filter(x => x.responseCode != 200)

// make a pair of (x, 1)
val endpointCountPairTuple = not200.map(x => (x.endpoint, 1))

// count the number of records for each key x
val endpointSum = endpointCountPairTuple.reduceByKey(_ + _)

// take the top 10
val topTenErrURLs = endpointSum.takeOrdered(10)(Ordering[Int].on(x => x._2 * -1))

println("Top ten failed URLs: ")
topTenErrURLs.foreach(println)

Top ten failed URLs: 
(/images/NASA-logosmall.gif,8761)
(/images/KSC-logosmall.gif,7236)
(/images/MOSAIC-logosmall.gif,5197)
(/images/USA-logosmall.gif,5157)
(/images/WORLD-logosmall.gif,5020)
(/images/ksclogo-medium.gif,4728)
(/history/apollo/images/apollo-logo1.gif,2907)
(/images/launch-logo.gif,2811)
(/,2199)
(/images/ksclogosmall.gif,1622)


not200: org.apache.spark.rdd.RDD[Row] = MapPartitionsRDD[17] at filter at <console>:29
endpointCountPairTuple: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[18] at map at <console>:32
endpointSum: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[19] at reduceByKey at <console>:35
topTenErrURLs: Array[(String, Int)] = Array((/images/NASA-logosmall.gif,8761), (/images/KSC-logosmall.gif,7236), (/images/MOSAIC-logosmall.gif,5197), (/images/USA-logosmall.gif,5157), (/images/WORLD-logosmall.gif,5020), (/images/ksclogo-medium.gif,4728), (/history/apollo/images/apollo-logo1.gif,2907), (/images/launch-logo.gif,2811), (/,2199), (/images/ksclogosmall.gif,1622))


Let's count the number of unique hosts in the entire log. Think about the steps that you need to perform to count the number of different hosts in the log. The result should be as below:
```
Unique hosts: 54507

```

In [8]:
// extract the host field for each record
val hosts = accessLogs.map(x => x.host)

// keep the uniqe hosts
val uniqueHosts = hosts.distinct

// count them
val uniqueHostCount = uniqueHosts.count()

println("Unique hosts: " + uniqueHostCount)

Unique hosts: 54507


hosts: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[21] at map at <console>:31
uniqueHosts: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[24] at distinct at <console>:34
uniqueHostCount: Long = 54507


For an advanced exercise, let's determine the number of unique hosts in the entire log on a day-by-day basis. This computation will give us counts of the number of unique daily hosts. We'd like a list sorted by increasing day of the month, which includes the day of the month and the associated number of unique hosts for that day. Make sure you cache the resulting RDD `dailyHosts`, so that we can reuse it in the next exercise. Think about the steps that you need to perform to count the number of different hosts that make requests *each* day. Since the log only covers a single month, you can ignore the month. Here is the output you should receive:
```
Unique hosts per day: 
(1,2582)
(3,3222)
(4,4190)
(5,2502)
(6,2537)
(7,4106)
(8,4406)
(9,4317)
(10,4523)
(11,4346)
(12,2864)
(13,2650)
(14,4454)
(15,4214)
(16,4340)
(17,4385)
(18,4168)
(19,2550)
(20,2560)
(21,4134)
(22,4456)
```

In [10]:
// make pairs of (day, host)
val dayToHostPairTuple = accessLogs.map(x => (x.dateTime.day, x.host)).distinct()

// group by day
val dayGroupedHosts = dayToHostPairTuple.groupByKey

// make pairs of (day, number of host in that day)
val dayHostCount = dayGroupedHosts.map(x => (x._1 , x._2.size))

// sort by day
val dailyHosts = dayHostCount.sortByKey()

// return the records as a list
val dailyHostsList = dailyHosts.take(30)

println("Unique hosts per day: ")
dailyHostsList.foreach(println)

Unique hosts per day: 
(1,2582)
(3,3222)
(4,4190)
(5,2502)
(6,2537)
(7,4106)
(8,4406)
(9,4317)
(10,4523)
(11,4346)
(12,2864)
(13,2650)
(14,4454)
(15,4214)
(16,4340)
(17,4385)
(18,4168)
(19,2550)
(20,2560)
(21,4134)
(22,4456)


dayToHostPairTuple: org.apache.spark.rdd.RDD[(Int, String)] = MapPartitionsRDD[37] at distinct at <console>:35
dayGroupedHosts: org.apache.spark.rdd.RDD[(Int, Iterable[String])] = ShuffledRDD[38] at groupByKey at <console>:38
dayHostCount: org.apache.spark.rdd.RDD[(Int, Int)] = MapPartitionsRDD[39] at map at <console>:41
dailyHosts: org.apache.spark.rdd.RDD[(Int, Int)] = ShuffledRDD[42] at sortByKey at <console>:44
dailyHostsList: Array[(Int, Int)] = Array((1,2582), (3,3222), (4,4190), (5,2502), (6,2537), (7,4106), (8,4406), (9,4317), (10,4523), (11,4346), (12,2864), (13,2650), (14,4454), (15,4214), (16,4340), (17,4385), (18,4168), (19,2550), (20,2560), (21,4134), (22,4456))


Next, let's determine the average number of requests on a day-by-day basis. We'd like a list by increasing day of the month and the associated average number of requests per host for that day. Make sure you cache the resulting RDD `avgDailyReqPerHost` so that we can reuse it in the next exercise. To compute the average number of requests per host, get the total number of request across all hosts and divide that by the number of unique hosts. Since the log only covers a single month, you can skip checking for the month. Also to keep it simple, when calculating the approximate average use the integer value. The result should be as below:
```
Average number of daily requests per Hosts is: 
(1,13)
(3,12)
(4,14)
(5,12)
(6,12)
(7,13)
(8,13)
(9,14)
(10,13)
(11,14)
(12,13)
(13,13)
(14,13)
(15,13)
(16,13)
(17,13)
(18,13)
(19,12)
(20,12)
(21,13)
(22,12)
```

In [12]:
// make pairs of (day, host)
val dayAndHostTuple = accessLogs.map(x => (x.dateTime.day, x.host))

// group by day
val groupedByDay = dayAndHostTuple.groupByKey

// sort by day
val sortedByDay = groupedByDay.sortByKey()

// calculate the average request per day
val avgDailyReqPerHost = sortedByDay.map(x => (x._1, x._2.size/x._2.toSet.size))

// return the records as a list
val avgDailyReqPerHostList = avgDailyReqPerHost.take(30)

println("Average number of daily requests per Hosts is: ")
avgDailyReqPerHostList.foreach(println)

Average number of daily requests per Hosts is: 
(1,13)
(3,12)
(4,14)
(5,12)
(6,12)
(7,13)
(8,13)
(9,14)
(10,13)
(11,14)
(12,13)
(13,13)
(14,13)
(15,13)
(16,13)
(17,13)
(18,13)
(19,12)
(20,12)
(21,13)
(22,12)


dayAndHostTuple: org.apache.spark.rdd.RDD[(Int, String)] = MapPartitionsRDD[49] at map at <console>:35
groupedByDay: org.apache.spark.rdd.RDD[(Int, Iterable[String])] = ShuffledRDD[50] at groupByKey at <console>:38
sortedByDay: org.apache.spark.rdd.RDD[(Int, Iterable[String])] = ShuffledRDD[53] at sortByKey at <console>:41
avgDailyReqPerHost: org.apache.spark.rdd.RDD[(Int, Int)] = MapPartitionsRDD[54] at map at <console>:44
avgDailyReqPerHostList: Array[(Int, Int)] = Array((1,13), (3,12), (4,14), (5,12), (6,12), (7,13), (8,13), (9,14), (10,13), (11,14), (12,13), (13,13), (14,13), (15,13), (16,13), (17,13), (18,13), (19,12), (20,12), (21,13), (22,12))


### Exploring 404 Response Codes
Let's count the 404 response codes. Create a RDD containing only log records with a 404 response code. Make sure you `cache()` the RDD `badRecords` as we will use it in the rest of this exercise. How many 404 records are in the log? Here is the result:
```
Found 6185 404 URLs.
```

In [13]:
val badRecords = accessLogs.filter(x => x.responseCode == 404).cache()

println("Found " + badRecords.count() + " 404 URLs.")

Found 6185 404 URLs.


badRecords: org.apache.spark.rdd.RDD[Row] = MapPartitionsRDD[55] at filter at <console>:28


Now, let's list the 404 response code records. Using the RDD containing only log records with a 404 response code that you cached in the previous part, print out a list up to 10 distinct endpoints that generate 404 errors -  no endpoint should appear more than once in your list. You should receive the follwoing output as your result:
```
404 URLS: 
/SHUTTLE/COUNTDOWN
/shuttle/missions/sts-71/images/www.acm.uiuc.edu/rml/Gifs
/shuttle/technology/stsnewsrof/stsref-toc.html
/de/systems.html
/ksc.htnl
/~pccomp/graphics/sinsght.gif
/PERSONS/NASA-CM.
/shuttle/missions/sts-1/sts-1-mission.html
/history/apollo/sa-1/sa-1-patch-small.gif
/images/sts-63-Imax
```

In [15]:
val badEndpoints = badRecords.map(x => x.endpoint)
val badUniqueEndpoints = badEndpoints.distinct
val badUniqueEndpointsPick10 = badUniqueEndpoints.take(10)

println("404 URLS: ")
badUniqueEndpointsPick10.foreach(println)

404 URLS: 
/SHUTTLE/COUNTDOWN
/shuttle/missions/sts-71/images/www.acm.uiuc.edu/rml/Gifs
/shuttle/technology/stsnewsrof/stsref-toc.html
/de/systems.html
/ksc.htnl
/~pccomp/graphics/sinsght.gif
/PERSONS/NASA-CM.
/shuttle/missions/sts-1/sts-1-mission.html
/history/apollo/sa-1/sa-1-patch-small.gif
/images/sts-63-Imax


badEndpoints: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[60] at map at <console>:32
badUniqueEndpoints: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[63] at distinct at <console>:33
badUniqueEndpointsPick10: Array[String] = Array(/SHUTTLE/COUNTDOWN, /shuttle/missions/sts-71/images/www.acm.uiuc.edu/rml/Gifs, /shuttle/technology/stsnewsrof/stsref-toc.html, /de/systems.html, /ksc.htnl, /~pccomp/graphics/sinsght.gif, /PERSONS/NASA-CM., /shuttle/missions/sts-1/sts-1-mission.html, /history/apollo/sa-1/sa-1-patch-small.gif, /images/sts-63-Imax)


Using the RDD containing only log records with a 404 response code that you cached before, print out a list of the top 10 endpoints that generate the most 404 errors. Remember, top endpoints should be in sorted order. The result would be as below:
```
Top ten 404 URLs: 
(/pub/winvn/readme.txt,633)
(/pub/winvn/release.txt,494)
(/shuttle/missions/STS-69/mission-STS-69.html,431)
(/images/nasa-logo.gif,319)
(/elv/DELTA/uncons.htm,178)
(/shuttle/missions/sts-68/ksc-upclose.gif,156)
(/history/apollo/sa-1/sa-1-patch-small.gif,146)
(/images/crawlerway-logo.gif,120)
(/://spacelink.msfc.nasa.gov,117)
(/history/apollo/pad-abort-test-1/pad-abort-test-1-patch-small.gif,100)
```

In [16]:
val badEndpointsCountPairTuple = badRecords.map(x => (x.endpoint, 1))
val badEndpointsSum = badEndpointsCountPairTuple.reduceByKey(_ + _)

// sort the records (you should implement a new object by extending the Ordering class for this part)
object ErrOrdering extends Ordering[(String, Int)] {
  def compare(x: (String, Int), y: (String, Int)) = x._2 compare y._2
}

val badEndpointsTop10 = badEndpointsSum.takeOrdered(10)(ErrOrdering.reverse)

println("Top ten 404 URLs: ")
badEndpointsTop10.foreach(println)

Top ten 404 URLs: 
(/pub/winvn/readme.txt,633)
(/pub/winvn/release.txt,494)
(/shuttle/missions/STS-69/mission-STS-69.html,431)
(/images/nasa-logo.gif,319)
(/elv/DELTA/uncons.htm,178)
(/shuttle/missions/sts-68/ksc-upclose.gif,156)
(/history/apollo/sa-1/sa-1-patch-small.gif,146)
(/images/crawlerway-logo.gif,120)
(/://spacelink.msfc.nasa.gov,117)
(/history/apollo/pad-abort-test-1/pad-abort-test-1-patch-small.gif,100)


badEndpointsCountPairTuple: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[64] at map at <console>:28
badEndpointsSum: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[65] at reduceByKey at <console>:29
defined object ErrOrdering
badEndpointsTop10: Array[(String, Int)] = Array((/pub/winvn/readme.txt,633), (/pub/winvn/release.txt,494), (/shuttle/missions/STS-69/mission-STS-69.html,431), (/images/nasa-logo.gif,319), (/elv/DELTA/uncons.htm,178), (/shuttle/missions/sts-68/ksc-upclose.gif,156), (/history/apollo/sa-1/sa-1-patch-small.gif,146), (/images/crawlerway-logo.gif,120), (/://spacelink.msfc.nasa.gov,117), (/history/apollo/pad-abort-test-1/pad-abort-test-1-patch-small.gif,100))


Instead of looking at the endpoints that generated 404 errors, now let's look at the hosts that encountered 404 errors. Using the RDD containing only log records with a 404 response code that you cached before, print out a list of the top 10 hosts that generate the most 404 errors. Here is the result:
```
Top ten hosts that generated errors: 
(piweba3y.prodigy.com,39)
(maz3.maz.net,39)
(gate.barr.com,38)
(m38-370-9.mit.edu,37)
(ts8-1.westwood.ts.ucla.edu,37)
(nexus.mlckew.edu.au,37)
(204.62.245.32,33)
(spica.sci.isas.ac.jp,27)
(163.206.104.34,27)
(www-d4.proxy.aol.com,26)
```

In [17]:
val errHostsCountPairTuple = badRecords.map(x => (x.host, 1))
val errHostsSum = errHostsCountPairTuple.reduceByKey(_ + _)
val errHostsTop10 = errHostsSum.takeOrdered(10)(ErrOrdering.reverse)

println("Top ten hosts that generated errors: ")
errHostsTop10.foreach(println)

Top ten hosts that generated errors: 
(piweba3y.prodigy.com,39)
(maz3.maz.net,39)
(gate.barr.com,38)
(nexus.mlckew.edu.au,37)
(ts8-1.westwood.ts.ucla.edu,37)
(m38-370-9.mit.edu,37)
(204.62.245.32,33)
(163.206.104.34,27)
(spica.sci.isas.ac.jp,27)
(www-d4.proxy.aol.com,26)


errHostsCountPairTuple: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[67] at map at <console>:30
errHostsSum: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[68] at reduceByKey at <console>:31
errHostsTop10: Array[(String, Int)] = Array((piweba3y.prodigy.com,39), (maz3.maz.net,39), (gate.barr.com,38), (nexus.mlckew.edu.au,37), (ts8-1.westwood.ts.ucla.edu,37), (m38-370-9.mit.edu,37), (204.62.245.32,33), (163.206.104.34,27), (spica.sci.isas.ac.jp,27), (www-d4.proxy.aol.com,26))


Let's explore the 404 records temporally. Break down the 404 requests by day and get the daily counts sorted by day as a list. Since the log only covers a single month, you can ignore the month in your checks. Cache the `errDateSorted` at the end. The output should be as below:
```
404 errors by day: 
(1,243)
(3,303)
(4,346)
(5,234)
(6,372)
(7,532)
(8,381)
(9,279)
(10,314)
(11,263)
(12,195)
(13,216)
(14,287)
(15,326)
(16,258)
(17,269)
(18,255)
(19,207)
(20,312)
(21,305)
(22,288)
```

In [18]:
val errDateCountPairTuple = badRecords.map(x => (x.dateTime.day, 1))
val errDateSum = errDateCountPairTuple.reduceByKey(_ + _)
val errDateSorted = errDateSum
val errByDate = errDateSorted.takeOrdered(30)
errDateSorted.cache()

println("404 errors by day: ")
errByDate.foreach(println)

404 errors by day: 
(1,243)
(3,303)
(4,346)
(5,234)
(6,372)
(7,532)
(8,381)
(9,279)
(10,314)
(11,263)
(12,195)
(13,216)
(14,287)
(15,326)
(16,258)
(17,269)
(18,255)
(19,207)
(20,312)
(21,305)
(22,288)


errDateCountPairTuple: org.apache.spark.rdd.RDD[(Int, Int)] = MapPartitionsRDD[70] at map at <console>:28
errDateSum: org.apache.spark.rdd.RDD[(Int, Int)] = ShuffledRDD[71] at reduceByKey at <console>:29
errDateSorted: org.apache.spark.rdd.RDD[(Int, Int)] = ShuffledRDD[71] at reduceByKey at <console>:29
errByDate: Array[(Int, Int)] = Array((1,243), (3,303), (4,346), (5,234), (6,372), (7,532), (8,381), (9,279), (10,314), (11,263), (12,195), (13,216), (14,287), (15,326), (16,258), (17,269), (18,255), (19,207), (20,312), (21,305), (22,288))


Using the RDD `errDateSorted` you cached before, what are the top five days for 404 response codes and the corresponding counts of 404 response codes?
```
Top five dates for 404 requests: (7,532) (8,381) (6,372) (4,346) (15,326) 
```

In [19]:
object DateOrdering extends Ordering[(Int, Int)] {
  def compare(x: (Int, Int), y: (Int, Int)) = x._2 compare y._2
}

val topErrDate = errDateSorted.takeOrdered(5)(DateOrdering.reverse)

print("Top five dates for 404 requests: ")
topErrDate.foreach(x => print(x + " "))

Top five dates for 404 requests: (7,532) (8,381) (6,372) (4,346) (15,326) 

defined object DateOrdering
topErrDate: Array[(Int, Int)] = Array((7,532), (8,381), (6,372), (4,346), (15,326))


Using the RDD `badRecords` you cached before, and by hour of the day and in increasing order, create an RDD containing how many requests had a 404 return code for each hour of the day (midnight starts at 0). 
```
Top hours for 404 requests: 
(0,175)
(1,171)
(2,422)
(3,272)
(4,102)
(5,95)
(6,93)
(7,122)
(8,199)
(9,185)
(10,329)
(11,263)
(12,438)
(13,397)
(14,318)
(15,347)
(16,373)
(17,330)
(18,268)
(19,269)
(20,270)
(21,241)
(22,234)
(23,272)
```

In [20]:
val hourCountPairTuple = badRecords.map(x => (x.dateTime.hour, 1))
val hourRecordsSum = hourCountPairTuple.reduceByKey(_ + _)
val hourRecordsSorted = hourRecordsSum.sortByKey()
val errHourList = hourRecordsSorted.collect()

println("Top hours for 404 requests: ")
errHourList.foreach(println)

Top hours for 404 requests: 
(0,175)
(1,171)
(2,422)
(3,272)
(4,102)
(5,95)
(6,93)
(7,122)
(8,199)
(9,185)
(10,329)
(11,263)
(12,438)
(13,397)
(14,318)
(15,347)
(16,373)
(17,330)
(18,268)
(19,269)
(20,270)
(21,241)
(22,234)
(23,272)


hourCountPairTuple: org.apache.spark.rdd.RDD[(Int, Int)] = MapPartitionsRDD[74] at map at <console>:28
hourRecordsSum: org.apache.spark.rdd.RDD[(Int, Int)] = ShuffledRDD[75] at reduceByKey at <console>:29
hourRecordsSorted: org.apache.spark.rdd.RDD[(Int, Int)] = ShuffledRDD[78] at sortByKey at <console>:30
errHourList: Array[(Int, Int)] = Array((0,175), (1,171), (2,422), (3,272), (4,102), (5,95), (6,93), (7,122), (8,199), (9,185), (10,329), (11,263), (12,438), (13,397), (14,318), (15,347), (16,373), (17,330), (18,268), (19,269), (20,270), (21,241), (22,234), (23,272))
